**Async: a style of concurrent programming in which tasks release the CPU during waiting periods, so that other tasks can use it**


3 ways of doing multiple things at once in Python: multiprocessing, multithreading, asynchronous programming

**Multiprocessing and multithreading**

Concurrency is when two or more tasks can start, run, and complete in overlapping time periods. It doesn't necessarily mean they'll ever both be running at the same instant. Eg. multitasking on a single-core machine. Parallelism is when two or more tasks  run at the same time, e.g. on a multicore processor. Concurrency can be characterized as a property of a program or system and parallelism as the run-time behaviour of executing multiple tasks at the same time. 

![](images/concurrency_parallelism.png)
If we ran this program on a computer with a single CPU core, the OS would be switching between the two threads, allowing one thread to run at a time. If we ran this program on a computer with a multi-core CPU then we would be able to run the two threads in parallel - side by side at the exact same time

A thread is a sequence of instructions within a process. It can be thought of as a lightweight process. Threads share the same memory space. A process is an instance of a program running in a computer which can contain one or more threads. A process has its independant memory space. 
    
The threading module is used for working with threads in Python. The CPython implementation has a Global Interpreter Lock (GIL) which allows only one thread to be active in the interpreter at once. The mechanism used by the CPython interpreter to assure that only one thread executes Python bytecode at a time. This simplifies the CPython implementation by making the object model (including critical built-in types such as dict) implicitly safe against concurrent access. Locking the entire interpreter makes it easier for the interpreter to be multi-threaded, at the expense of much of the parallelism afforded by multi-processor machines. This means that threads cannot be used for parallel execution of Python code. While parallel CPU computation is not possible, parallel IO operations are possible using threads. This is because performing IO operations releases the GIL. What are threads used for in Python? 

    In GUI applications to keep the UI thread responsive

    IO tasks (network IO or filesystem IO)

Using threads for these tasks improves performance, since in network IO for example, most of the time is spent waiting for a response from the URL. Threads should not be used for CPU bound tasks as this will actually result in worse performance compared to using a single thread.

For parallel execution of tasks use multiprocessing, a package that supports spawning processes using an API similar to the threading module. It side-steps the GIL by using subprocesses instead of threads. The Pool object which offers a convenient means of parallelizing the execution of a function across multiple input values, distributing the input data across processes (data parallelism).

**Using coroutines to run functions concurrently**

While threads are useful for blocking I/O, they should be avoided for parallel procesessing. Three problems with threads: special tools are needed e.g. to prevent data races, which increases code complexity and difficultly with its maintenance, and executing threads requires uses a lot of memory and are costly to start. 

Python work around these rpoblem with coroutines. These allow you to have many seemingly simultaneous functions running in your programs. They are are extension of generators and the cost of starting them is just a function call and require only a small amount of memory until they are exhausted. 

Again, how does Python do multiple things at once?

1 - Multiple processes

The most obvious way is to use multiple processes. From the terminal you can start your script two, three, four…ten times and then all the scripts are going to run independently or at the same time. The operating system that's underneath will take care of sharing your CPU resources among all those instances. Using CPython that's actually the only way you can get to use more than one CPU at the same time.

2 - Multiple threads

The next way to run multiple things at once is to use threads.

A thread is a line of execution, pretty much like a process, but you can have multiple threads in the context of one process and they all share access to common resources. But because of this it's difficult to write a threading code. And again, the operating system is doing all the heavy lifting on sharing the CPU, but the global interpreter lock (GIL) allows only one thread to run Python code at a given time even when you have multiple threads running code. So, In CPython, the GIL prevents multi core concurrency. Basically, you’re running in a single core even though you may have two or four or more.

3 - Asynchronous Programming

The third way is an asynchronous programming, where the OS is not participating. As far as OS is concerned you're going to have one process and there's going to be a single thread within that process, but you'll be able to do multiple things at once. So, what's the trick? 

Much of the code we write, especially in heavy IO applications like websites, depends on external resources. This could be anything from a remote database call to POSTing to a REST service. As soon as you ask for any of these resources, your code is waiting around with nothing to do. With asynchronous programming, you allow your code to handle other tasks while waiting for these other resources to respond.

In [54]:
def my_gen():
    for i in range(10):
        if i % 2 == 0:
            yield i
        
g = my_gen()
g.send(None)

0

Coroutines: a background
-------------------------

Coroutines have similarities to generators. When you call a generator function, a generator object is returned. The function is not run, it only executes on `next()`. Yield produces a value but the function is suspended and resumes on the next call to `next()` until StopIteration is reached.

Following PEP-342, you can use `yield` in expressions, such as the RHS of assignments. If you use `yield` more generally, you get a coroutine. These do more than generate values, they can consume values sent to them (you can only send None to a generator, effectively the same as calling `next()` on it). Values sent to them are returned by (yield):

In [59]:
def grep(pattern):
    print ("Looking for {}".format(pattern))
    while True:        
        line = (yield)       
        if pattern in line:            
            print(line)
            
g = grep('Python')
next(g)

Looking for Python


Execution is similar as for generators. 'Priming' the coroutine by calling `next()` or `.send(None)` advances the coroutine to the first `yield` and execution is suspended until it is sent a value. 

In [60]:
g.send('no snakes here')
g.send('Python, you are so cool')
g.close()

Python, you are so cool


In [56]:
#The need to call next()`can be solved using a decorator :

def coroutine(func):    
    def start(*args,**kwargs):        
        cr = func(*args,**kwargs)        
        next(cr)        
        return cr    
    return start

**Processing Pipelines**

Coroutines can be used to set up pipes. You just chain coroutines together and push data through the pipe with `send()` operations

![](images/proc_pipeline.png)

The source (typically not a coroutine) drives the pipeline. You can have branching and arbitrarily complex routing of data: 

![](images/pipeline_branch.png)

Coroutines provide more powerful data routing possibilities than simple iterators. If you built a collection of simple data processing components, you can glue them together into complex arrangements of pipes, branches, merging, etc. 

In [15]:
import asyncio

async def speak_async():  
    print('OMG asynchronicity!')

# cannot run with something driving it
speak_async()

<coroutine object speak_async at 0x0000019E7B8DA780>

In [ ]:
async def speak_async():  
    print('OMG asynchronicity!')
    
async def run_this()

In [61]:
loop = asyncio.get_event_loop()  
loop.run_until_complete(speak_async())  
loop.close()  

RuntimeError: Event loop is closed